# Implementación Bloom filter


In [ ]:
import math
import numpy as np
import random

class BloomFilter:

  def __init__(self, p, dataset):

    # Comprueba si el valor de p se encuentra dentro del intervalo correcto
    if p <= 0 or p >= 1:
        raise ValueError("La probabilidad de falso positivo debe estar entre 0 y 1")

    self.dataset = dataset

    # Longitud del dataset
    self.n = len(dataset)

    # Longitud del array. Se elige el entero más cercano a la aproximación de m
    self.m = int(- self.n * math.log(p) / math.log(2)**2)

    # Cantidad de funciones hash. Se elige el entero más cercano a la aproximación de k
    self.k = int(- math.log(p) / math.log(2))

    # Se crea una familia de k funciones a partir del método hash_function, que por su construcción garantiza que la familia sea k-independiente
    self.hash_family = [self.hash_function() for _ in range(self.k)]

    # Se crea una array de tamaño m inicializado en cero
    self.B = np.zeros(self.m, int)


  # Se actualiza el array contando todos los elementos del dataset
  def update(self):

    for element in self.dataset:

      for j in range(self.k):

        # Para poder aplicar las funciones creadas con hash_function, el elemento debe estar en formato numérico,
        # luego se aplica hash (que devuelve un hash del elemento positivo o negativo) y se le aplica el valor absoluto
        self.B[self.hash_family[j](abs(hash(element)))] = 1


  # Se comprueba si un elemento pertenece al dataset
  def membership(self, element):

    for j in range(self.k):

        # Si al aplicar la función hash el bit del array es cero, entonces el elemento no existe en el dataset
        if self.B[self.hash_family[j](abs(hash(element)))] == 0:

          return "El elemento '" + str(element) + "' no se encuentra en el dataset proporcionado."

        return "El elemento '" + str(element) + "' se encuentra en el dataset proporcionado."



  # Este método devuelve una función
  def hash_function(self):

    # Se elige un número primo grande aleatorio p, por ejemplo, el siguiente
    p = 2**61 - 1

    # Se eligen k enteros de forma aleatoria entre 0 y el número primo p
    a = [random.randint(0, p) for _ in range(self.k)]

    # Se devuelve una función que dado un elemento (en forma numérica) devuelve un entero entre 0 y m-1
    return lambda element: sum([a[i]*element**i for i in range(self.k)]) % p % self.m


Importamos el dataset Airline Passenger Satisfaction de Kaggle.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d teejmahal20/airline-passenger-satisfaction
!unzip airline-passenger-satisfaction.zip

  0% 0.00/2.71M [00:00<?, ?B/s]
100% 2.71M/2.71M [00:00<00:00, 143MB/s]
Archive:  airline-passenger-satisfaction.zip
  inflating: test.csv                
  inflating: train.csv               


Los datos están divididos en dos grupos, en este caso usaremos train.csv para comprobar que el algoritmo anterior funciona correctamente. Para ello, comprobamos previamente si existe alguna edad en particular dentro del grupo Age.

In [ ]:
import pandas as pd
data = pd.read_csv('train.csv')
edad = 13
if edad in data['Age'].values:
  print('Existe alguien con edad ' + str(edad) + ' en el dataset.')
else:
  print('No existe alguien con edad ' + str(edad) + ' en el dataset.')

Existe alguien con edad 13 en el dataset.


A continuación, creamos un objeto de clase BloomFilter, establecemos $p = 0.01$, definimos el dataset, actualizamos el array con los datos y comprobamos si un elemento pertenece o no al dataset.

In [ ]:
bf = BloomFilter(p=0.01, dataset=data['Age'])

In [ ]:
bf.update()

In [ ]:
bf.membership(13)

"El elemento '13' se encuentra en el dataset proporcionado."

Vemos que estima correctamente que el elemento se encuentra en el dataset. Si comprobamos una edad que no está (por ejemplo, una edad de 500 años sería imposible), dará como resultado que no se encuentra en el dataset.

In [ ]:
bf.membership(500)

"El elemento '500' no se encuentra en el dataset proporcionado."